In [1]:
import polars as pl
import tiktoken
import os
import numpy as np
from pathlib import Path
from ast import literal_eval

In [3]:
number_return_values = 1

In [4]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-HSrHGyOHtEDtr9loLzRjT3BlbkFJTUFaqCLYDEs3B7qpBz7z"))

In [5]:
transcript_and_translation_json= Path().cwd().parent.joinpath('data/output_json/transcript_and_translation_1.json')

In [6]:
transcript_and_translation_df = pl.read_json(transcript_and_translation_json)

In [7]:
transcript_and_translation_df.head(5)

shape: (5, 3)
┌────────────┬─────────────────────────────┬─────────────────────────────────┐
│ callsign   ┆ file_name                   ┆ transcription                   │
│ ---        ┆ ---                         ┆ ---                             │
│ str        ┆ str                         ┆ list[str]                       │
╞════════════╪═════════════════════════════╪═════════════════════════════════╡
│ 2024-05-07 ┆ 2024-05-07_00-21-36_000.mp3 ┆ ["Otavola es un tipo, como te … │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_001.mp3 ┆ ["How about a quarter to ten? … │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_002.mp3 ┆ ["Ahora, de ahora en adelante,… │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_003.mp3 ┆ ["Votación, sube la música", "… │
│ 2024-05-07 ┆ 2024-05-07_00-21-36_004.mp3 ┆ ["Oh, oh, who's this? Da, di, … │
└────────────┴─────────────────────────────┴─────────────────────────────────┘

In [9]:
exploded_transcript_df = transcript_and_translation_df.explode('transcription')
exploded_transcript_df = exploded_transcript_df[['file_name', 'transcription']]

In [10]:
exploded_transcript_df

file_name,transcription
str,str
"""2024-05-07_00-21-36_000.mp3""","""Otavola es un tipo, como te lo…"
"""2024-05-07_00-21-36_000.mp3""","""Por suerte, ya eso pasó"""
"""2024-05-07_00-21-36_000.mp3""","""Gracias, Otavola, pues si no l…"
"""2024-05-07_00-21-36_000.mp3""","""Lo de Caído no lo sabía"""
"""2024-05-07_00-21-36_000.mp3""","""Yo creo que hiciste bien decir…"
…,…
"""2024-05-07_00-21-36_005.mp3""","""En la Ford de Kindle hay dulce…"
"""2024-05-07_00-21-36_005.mp3""","""Y eso no es todo, ¿eh? Porque …"
"""2024-05-07_00-21-36_005.mp3""","""Así que, si está buscando un L…"


In [11]:
len(exploded_transcript_df)

631

In [13]:
embedded_false_statements_path= Path().cwd().parent.joinpath('data/factchecked_statements/embedded_false_statements.csv')

In [50]:
false_text_df = pl.read_csv(embedded_false_statements_path)

In [51]:
false_text_df['statement_embedding']

statement_embedding
str
"""[0.01533014141023159, -0.00409…"
"""[0.0013132952153682709, -0.030…"
"""[-0.007365651428699493, 0.0279…"
"""[0.028059430420398712, -0.0474…"
"""[-0.0042332131415605545, -0.00…"
…
"""[-0.033378954976797104, -0.002…"
"""[-0.04214097559452057, 0.02756…"
"""[0.019191721454262733, 0.00352…"


In [52]:
false_text_dict = false_text_df.to_dict()

In [53]:
statement_embedding_numpy = false_text_df['statement_embedding'].to_numpy()

In [54]:
false_text_df = false_text_df.with_columns(
    pl.Series(name='statement_embedding_numpy', values=statement_embedding_numpy))

In [55]:
false_text_df['statement_embedding_numpy'][0]

'[0.01533014141023159, -0.004097426310181618, -0.026958515867590904, 0.051395539194345474, 0.04450166970491409, 0.009596426971256733, 0.01860271766781807, 0.04176558181643486, -0.017583390697836876, 0.036910366266965866, -0.004650679416954517, 0.02344452030956745, 0.011500959284603596, 0.009261121973395348, -0.002452755579724908, 0.03256481513381004, -0.01191673707216978, 0.0017284968635067344, -0.03589104115962982, 0.02188870497047901, 0.018562480807304382, 0.03876125067472458, -0.009837846271693707, 0.020963264629244804, -0.026529325172305107, -0.019850051030516624, 0.00371517869643867, 0.0195013340562582, -0.03066028282046318, -0.028863048180937767, 0.013311605900526047, -0.03605198487639427, 0.013620086014270782, -0.051905203610658646, 0.0005712757701985538, 0.011286363936960697, -0.01274158712476492, -0.02787054516375065, 0.03184055536985397, -0.022143537178635597, 0.0079668452963233, -0.06523692607879639, 0.015625210478901863, 0.06738287955522537, 0.006377499550580978, -0.0431604

In [56]:
# false_text_df = false_text_df.with_columns(false_text_df['statement'].replace('\'', '').replace('/"', '').replace('\"', '').replace('.','').replace('?','').replace('!',''))

In [57]:
false_text_df['statement'][8]

'Says Raphael Warnock has "said we need to open up the prisons" and "eliminate cash bail."'

In [58]:
def get_embedding(text, model="text-embedding-3-small"):
   # text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [59]:
def cosine_similarity(vec1, vec2):
    # Normalize each vector to unit length
    vec1_norm = vec1 / np.linalg.norm(vec1)
    vec2_norm = vec2 / np.linalg.norm(vec2)
  
    # Calculate dot product between normalized vectors
    similarity = np.dot(vec1_norm, vec2_norm)
    return similarity

def search_false_statements(search_terms,false_text_df=false_text_df, n=number_return_values, pprint=True):
   embedding = get_embedding(search_terms, model='text-embedding-3-small')
   false_text_df['similarities'] = false_text_df.statement_embedding.apply(lambda x: cosine_similarity(x, embedding))
   results = (
      false_text_df.sort_values("similarities", ascending=False)
      .head(n)
   )
   if pprint:
      for r in results:
         print(r[:200])
         print()
   return results

In [60]:
top_matches_json = []

def search_all_transcripts():
    for row in exploded_transcript_df.itertuples(name='segment'):
        results = search_false_statements(row.transcription, pprint=False)
        top_match = results.statement.values
        similarity = results.similarities
        match_index = row.Index
        top_matches = {}
        top_matches["index"] = f"{match_index}"
        top_matches["filename"] = row.file_name
        top_matches["input_statement"] = row.transcription
        top_matches["checked_false_statement"] = f"{top_match}"
        top_matches["similarity"] = f"{similarity}"
        top_matches_json.append(top_matches)
        print(f"Finished checking {match_index + 1} of {len(exploded_transcript_df)}")


In [61]:
search_all_transcripts()

AttributeError: 'DataFrame' object has no attribute 'itertuples'

In [ ]:
top_matches_json[0]

In [ ]:
top_matches_df = pl.DataFrame(top_matches_json)

In [ ]:
top_matches_df.sort_values('similarity', ascending=False)

In [ ]:
from libretranslatepy import LibreTranslateAPI

lt = LibreTranslateAPI("http://localhost:5000")

In [ ]:
def libretranslate_spanish(input_text):
    return lt.translate(f"{input_text}", "es", "en")

def libretranslate_french(input_text):
    return lt.translate(f"{input_text}", "fr", "en")

In [ ]:
libretranslate_spanish("hola, como estas")

In [ ]:
top_matches_df['similarity_value'] = top_matches_df['similarity'].apply(lambda x: x.split()[1]).astype(float)
top_matches_df['factcheck_index'] = top_matches_df['similarity'].apply(lambda x: x.split()[0])

In [ ]:
over_50 = top_matches_df.loc[top_matches_df['similarity_value'] >= .45]

In [ ]:
over_50['translation'] = over_50['input_statement'].apply(lambda x: libretranslate_french(x))
over_50['translation'] = over_50['translation'].apply(lambda x: libretranslate_spanish(x))

In [ ]:
over_50 = over_50.sort_values('similarity_value', ascending=False)


In [ ]:
over_50 = over_50[['filename', 'input_statement', 'translation', 'checked_false_statement', 'similarity_value', 'factcheck_index']]

In [ ]:
len(over_50)

In [ ]:
over_50.to_csv("over_50.csv")

In [ ]:
over_50